In [733]:
import matplotlib as mpl
# ^^^ pyforest auto-imports - don't write above this line
import os
import pandas as pd
import numpy as np
import sklearn.tree
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

sns.set_palette('Set2') #set colorblind-friendly palette for seaborn
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 1000000000)
mpl.rc('figure', max_open_warning = 0)
pd.options.display.float_format = '{:.2f}'.format
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

# Lag Analysis for Suite NPS
- I am going to be analyzing the results of six time lags on Suite NPS
- Suite NPS will be regressed on each App's NPS (PP, Excel, Outlook, Word), as well as subtheme's NPS (i.e. 'ease of use')
- Each subtheme will be condensed to its root, which will result in a final slice of subthemes accounting for all subtheme tags
    - weighted averages will be computed to account for combinations of subthemes
    
- __TODO: Update the overall analysis with the code for selecting only the single best model every week, code is included in individual app analysis__

In [1446]:
#read in data, data since January 2019
for file in os.listdir('C:\\Users\\fulto\\Desktop\\Lag NPS Analysis'):
    
    if file.startswith('weekly_app'):
        
        app = pd.read_csv(file)
    
    elif file.startswith('weekly_suite'):
        
        suite = pd.read_csv(file)
        
    elif file.startswith('weekly_subtheme'):
        
        subs = pd.read_csv(file).fillna('["not mapped"]')

In [1447]:
#format the app dataframe

#keep only these four apps
keep_these_apps = ['Excel', 'PowerPoint', 'Outlook', 'Word']

#subset df for these four
app = app[app['Feedback_App'].isin(keep_these_apps)]

#group df by week and nps
app = pd.DataFrame(app.groupby(['Feedback_App', 'Week']).avg_NPS.mean()).reset_index()

#create subsets to concatenate - need to reset axes and rename columns for concatenation to work properly
excel = app.iloc[:80, :].rename(columns = {'Feedback_App':'feedback_app_excel', 'avg_NPS':'excel_nps'}).drop('Week', axis = 1).reset_index(drop = True)
outlook = app.iloc[80:160, :].rename(columns = {'Feedback_App':'feedback_app_outlook', 'avg_NPS':'outlook_nps'}).drop('Week', axis = 1).reset_index(drop = True) 
pp = app.iloc[160:240, :].rename(columns = {'Feedback_App':'feedback_app_pp', 'avg_NPS':'pp_nps'}).drop('Week', axis = 1).reset_index(drop = True)
word = app.iloc[240:, :].rename(columns = {'Feedback_App':'feedback_app_word', 'avg_NPS':'word_nps'}).reset_index(drop = True)

#concatenate all these dfs into one
app = pd.concat([excel, outlook, pp, word], axis = 1)

#move week to the front of the df
app.insert(loc = 0, column = 'week', value = [x for x in app['Week']])

app.drop('Week', axis = 1, inplace = True)

#drop app name columns, just keep nps
app = app[['week', 'excel_nps', 'outlook_nps', 'pp_nps', 'word_nps']]

#now find the subthemes that stand on their own, that we we can combine the correct ones, this will essentially tell us how many columns we will have
themes = pd.DataFrame(subs.groupby(['Verbatim_Themes', 'Week', 'Count']).avg_NPS.mean())

themes.reset_index(inplace = True)

#there will always be one more element than the number of commas in a list, I only want elements from the column that have no commas
unique_subthemes = pd.DataFrame(themes[~themes['Verbatim_Themes'].str.contains(',')].Verbatim_Themes.unique())

#lowercase all columns in suite
suite.columns = [c.lower() for c in suite.columns]

#merge suite and app, clean it up a bit, dropping unnecessary columns
df = app.merge(suite, how = 'inner', on = 'week').rename(columns = {'avg_nps':'suite_nps'}).drop(['feedback_platform', 'count'], axis = 1)

#replace brackets surrounding elements in list of subtheme names, gotta do this in two stages I think (although I am sure someby knows a better way lol)
unique_subthemes = pd.DataFrame(unique_subthemes.iloc[:, 0].str.replace("[",  ''))

unique_subthemes = pd.DataFrame(unique_subthemes.iloc[:, 0].str.replace(']',  ''))

#remove quotation marks too
unique_subthemes = pd.DataFrame(unique_subthemes.iloc[:, 0].str.replace('"', ''))

#make this into a list so we can use the remove operation of lists
unique_subthemes = [x for x in unique_subthemes.iloc[:, 0]]


#create list of subthemes to remove from this list (we will be adding these into the weighted average) (yammer only had like 11 entries)
to_remove = ['easeofuse-howto', 'onedrive for business', 'easeofuse-version', 'not mapped', 'skype for business', 'yammer', 'intune', 'si communication']

#remove each element using list.remove()
for element in to_remove:
    
    unique_subthemes.remove(element)
    

#create dfs for all subthemes
def subset_df(theme):
    theme_sub = subs[subs['Verbatim_Themes'].str.contains(theme)]
    return theme_sub

#place each df into a dictionary under its own name, we can subset the dict to access dfs now
subdict = {}

for theme in unique_subthemes:
    
    subdict[theme] = pd.DataFrame(subset_df(theme))

def build_weighted_avgs(subtheme):
    
    '''
    
    This function takes a subtheme as an arg from the list of unique subthemes created previously, it creates a column of weighted average nps for each 
    week in each subtheme
    
    args: subtheme - meant to be used in a loop to create several different dfs
    
    '''
    
    df = pd.DataFrame(subdict[subtheme].groupby(['Verbatim_Themes', 'Week', 'Count']).avg_NPS.mean()).sort_values(['Week', 'Verbatim_Themes']).reset_index()
    
    all_weeks = [week for week in df['Week'].unique()]
    
    df['product'] = df['avg_NPS'] * df['Count']
    
    final_df = pd.DataFrame((pd.DataFrame(df.groupby('Week').product.sum())['product']) / 
                            (pd.DataFrame(df.groupby('Week').Count.sum())['Count'])).rename(columns = {0:f'{subtheme}_subtheme_nps'})
    
    return final_df

#similar to earlier, create a dict that will hold all of our dfs created with the function above, this will serve as a container for all of our new dfs, which we can then use to 
# concatenate them all into one df and eventually merge back into the master df
weighted_avgs = {}

for key, val in subdict.items():
    
    weighted_avgs[key] = build_weighted_avgs(key)

#just to give you an idea of what each one looks like
weighted_avg_dfs = []

for key, val in weighted_avgs.items():
    
    weighted_avg_dfs.append(val)
    
#concatenate them all (merging them together based on the index of Week, the index MUST be "Week" or else the df isn't right)
subs_weighted_nps = pd.concat(weighted_avg_dfs, axis = 1)

#now create final df for regression, concatenating the df we created with app and suite nps with this new subthemes nps df
ols_df = pd.concat([df.set_index('week'), subs_weighted_nps], axis = 1)


def results_summary_to_dataframe(results):
    
    '''
    
    take the result of an statsmodel results table and transforms it into a dataframe
    
    '''
    
    pvals = results.pvalues
    
    coeff = results.params
    
    conf_lower = results.conf_int()[0]
    
    conf_higher = results.conf_int()[1]

    results_df = pd.DataFrame({"pvals":pvals,
                               "coeff":coeff,
                               "conf_lower":conf_lower,
                               "conf_higher":conf_higher
                                })

    #Reordering...
    results_df = results_df[["coeff","pvals","conf_lower","conf_higher"]]
    
    return results_df

#function for simply lagging the df
def lag_df(df, lags):
    
    '''
    
    This function lags our df by every column except for suite nps
    
    '''
    
    lagged_df = pd.concat([df.loc[:, df.columns != 'suite_nps'].shift(lags), df['suite_nps']], axis = 1)
    
    return lagged_df

#just add a constant in right here for regression
ols_df['constant'] = 1

results_dfs = []

#create list of lags to iterate through
lags = list(range(0, 7, 1))

#create list of row ids to iterate through
rows = list(range(0, 80, 1))

#now forming a nested loop that will iterate first through the lagged df rows, then move on to running a regression for each row in that lagged df
# since regressions will not be able to be run for certain lag periods for certain rows, we are bypassing errors 
for row in rows:
    
    #iterate through list of lags
    for lag in lags:
    
        #try to execute the below code, if an error is encountered then pass it by
        try:
            
            #lag the df in steps, iterating through the list of lags (so 1 through 6)
            reg_df = lag_df(ols_df, lag)
        
            #create a df that is just one row of the ols_df to subset for regression
            regression_row = pd.DataFrame(reg_df.iloc[row, :]).T.dropna(axis = 1)
            
            #set the target to suite nps
            target = regression_row['suite_nps']
    
            #set features to every other column in that row
            features = regression_row.drop('suite_nps', axis = 1)
        
            #instantiate the model
            model = sm.OLS(target, features)
        
            #run the model and get results set
            results = model.fit()
        
            #send the results set to a dataframe
            ols_results = results_summary_to_dataframe(results)
            
            #create a column with the AIC (goodness of fit metric to compare to other models -- as AIC decreases, model gets better)
            ols_results['AIC'] = results.aic
            
            #create column for model's BIC, same as AIC for all intents and purposes
            ols_results['BIC'] = results.bic
            
            #create a tag to denote which lag the results set is from
            ols_results['lag_tag'] = lag
            
            #create tag to indicate which row out of the ols_df the regression was run on
            ols_results['row_tag'] = row
            
            #create week column that denotes the week of the data that the regression was run on -- this will be the week of the suite_nps value
            ols_results['week'] = ols_df.reset_index().iloc[row, 0]
            
            #append these results dfs to the list outside of the loop so that we can access them
            results_dfs.append(ols_results)
            
        #pass by errors    
        except:
        
            pass

def create_results_row(results_dfs, index):
    
    '''
    
    This function takes a results set df generated by the regression loop above and then makes the results fit onto one row, the goal
    here is to concatenate all of the rows eventually to have one whole results df containing all of the results for each lag and row
    
    args: results_dfs is the list of results dfs generated in the previous loop, index is the subset number, so this is the number of each individual
    df from the results dfs list, this function is meant to be looped over so the index is each number from 1-459
    
    '''
    
    #transpose the df 
    df = results_dfs[index].T
    
    #take this value and set it as a column, do the same for the next 4 things too
    df['aic'] = df.iloc[4, 0]
    
    df['bic'] = df.iloc[5, 0]
    
    df['lag'] = df.iloc[6, 0]
    
    df['suite_nps_row_num'] = df.iloc[7, 0]
    
    df['suite_nps_week'] = df.iloc[8, 0]
    
    #take the first row of the df that we created, transpose this as well to turn it into a single row df
    df = pd.DataFrame(df.iloc[0, :]).T
    
    return df

#here we are just running a loop that will go through every single index number of the results dfs (0 - 458 here) and use the function we created above
# to extract the results as a single row that we can use to concatenate to other rows like it to form a final results df

#create empty list to store rows we are creating
final_results = []

#create list of index values to use in subsetting the results df (0-458)
df_num = list(range(0, 539, 1))

#looping through our indices and appending the new rows to our list
for num in df_num:
    
    results_row = create_results_row(results_dfs, num)
    
    final_results.append(results_row)
    
#concatenating all of our new rows into one final df
final_results = pd.concat(final_results).reset_index(drop = True)

#format the final df how I want it, with week first
first_cols = final_results.iloc[:, 37:42]
first_cols = first_cols[first_cols.columns[::-1]]

#need to re-order columns
final_results = pd.concat([first_cols, final_results.drop(final_results.columns[37:42], axis = 1)], axis = 1)

#merge back in suite numbers
final_results = pd.merge(suite, final_results, how='left', left_on=['week'], right_on=['suite_nps_week']).sort_values(['suite_nps_row_num', 'lag'])\
            .rename(columns = {'avg_nps':'suite_nps', 'count':'suite_count', 'suite_nps_row_num':'week_block'}).drop(['suite_nps_week', 'feedback_platform'], axis = 1)

#make week datetime so it transfers to the csv correctly
final_results['week'] = pd.to_datetime(final_results['week'])

#send to csv
#final_results.to_csv('weekly_suite_lag_regression_analysis.csv', index = False, date_format='%Y-%m-%d')

## Potentially Using AIC and BIC Together With Coefficients to Determine Best Models
- I think that using the AIC and BIC may one (if not the only) way we can determine whether one model is better than another in this scenario/methodology
- Since we are running a regression on such little data and with only one single DV value, traditional statistics will not be able to be calculated properly (i.e. R^2). Due to this reality, I think it makes sense to use these fit criteria when they show a clear frontrunner(s). If AIC and BIC are the same for two models, then perhaps looking into the coefficients of apps as differentiators may be prudent. A summary of AIC and BIC is here:

AIC is an estimate of a constant plus the relative distance between the unknown true likelihood function of the data and the fitted likelihood function of the model, so that a lower AIC means a model is considered to be closer to the truth. BIC is an estimate of a function of the posterior probability of a model being true, under a certain Bayesian setup, so that a lower BIC means that a model is considered to be more likely to be the true model. Both criteria are based on various assumptions and asymptotic approximations. Each, despite its heuristic usefulness, has therefore been criticized as having questionable validity for real world data. But despite various subtle theoretical differences, their only difference in practice is the size of the penalty; BIC penalizes model complexity more heavily. The only way they should disagree is when AIC chooses a larger model than BIC.

AIC and BIC are both approximately correct according to a different goal and a different set of asymptotic assumptions. Both sets of assumptions have been criticized as unrealistic. Understanding the difference in their practical behavior is easiest if we consider the simple case of comparing two nested models. In such a case, several authors have pointed out that IC’s become equivalent to likelihood ratio tests with different alpha levels. Checking a chi-squared table, we see that AIC becomes like a significance test at alpha=.16, and BIC becomes like a significance test with alpha depending on sample size, e.g., .13 for n = 10, .032 for n = 100, .0086 for n = 1000, .0024 for n = 10000. Remember that power for any given alpha is increasing in n. Thus, AIC always has a chance of choosing too big a model, regardless of n. BIC has very little chance of choosing too big a model if n is sufficient, but it has a larger chance than AIC, for any given n, of choosing too small a model.

So what’s the bottom line? In general, it might be best to use AIC and BIC together in model selection. For example, in selecting the number of latent classes in a model, if BIC points to a three-class model and AIC points to a five-class model, it makes sense to select from models with 3, 4 and 5 latent classes. AIC is better in situations when a false negative finding would be considered more misleading than a false positive, and BIC is better in situations where a false positive is as misleading as, or more misleading than, a false negative.

# Running Same Analysis For Individual Apps
- Now, instead of running a regression on all of the columns, I will be running it only on single apps

In [1448]:
#prepping dfs, sorting values so that I can concatenate them
suite.sort_values('week', inplace = True)

app.sort_values('week', inplace = True)

app_lag = pd.concat([app, suite[['avg_nps', 'count']]], axis = 1)

app_lag.rename(columns = {'avg_nps':'suite_nps', 'count':'suite_count'}, inplace = True)

#modifying this function to pull only coefficients
def results_summary_to_dataframe(results):
    
    '''
    
    take the result of an statsmodel results table and transforms it into a dataframe
    
    '''
    
    
    
    coeff = results.params
    


    results_df = pd.DataFrame({
                               "coeff":coeff
                                })

    #Reordering...
    results_df = results_df[["coeff"]]
    
    return results_df

#setting up empty list to snatch up results dfs
app_summaries = []

#create list of lags to iterate through
lags = list(range(0, 7, 1))

#create list of row ids to iterate through
rows = list(range(0, 80, 1))

columns = ['excel_nps', 'outlook_nps', 'pp_nps', 'word_nps']

#now forming a nested loop that will iterate first through the lagged df rows, then move on to running a regression for each row in that lagged df
# since regressions will not be able to be run for certain lag periods for certain rows, we are bypassing errors 
for row in rows:
    
    for column in columns:
        
        try:
    
            #iterate through list of lags
            for lag in lags:
    
                #try to execute the below code, if an error is encountered then pass it by
            
                #lag the df in steps, iterating through the list of lags (so 1 through 6)
                reg_df = lag_df(app_lag, lag)
        
                #create a df that is just one row of the ols_df to subset for regression
                regression_row = pd.DataFrame(reg_df.iloc[row, :]).T.reset_index(drop = True)
            
                #set the target to suite nps
                target = regression_row['suite_nps']
    
                #set features to every other column in that row
                features = regression_row[column]
                features = pd.DataFrame(features).T.rename(columns = {0:column}).reset_index(drop = True)
            
                #add in constant
                features['constant'] = 1
        
                #instantiate the model
                app_model = sm.OLS(target.astype(float), features.astype(float))
        
                #run the model and get results set
                results = app_model.fit()
                
                app_ols_results = results_summary_to_dataframe(results)
            
                #create a column with the AIC (goodness of fit metric to compare to other models -- as AIC decreases, model gets better)
                app_ols_results['AIC'] = results.aic
            
                #create column for model's BIC, same as AIC for all intents and purposes
                app_ols_results['BIC'] = results.bic
            
                #create a tag to denote which lag the results set is from
                app_ols_results['lag'] = lag
            
                #create week column that denotes the week of the data that the regression was run on -- this will be the week of the suite_nps value
                app_ols_results['week_block'] = app_lag.reset_index().iloc[row, 0] + 1
            
                #append these results dfs to the list outside of the loop so that we can access them
                app_summaries.append(app_ols_results)
            
            #pass by errors    
        except:
        
            pass

#concqtenate all of the dfs in our list
summs = pd.concat(app_summaries)

#subset the dataframe, removing the constant coefficients
summs = summs[summs.index != 'constant'].reset_index().rename(columns = {'index':'app'})

#create a df with unique weeks and suite nps scores that we can merge back into our results df
week_df = pd.concat([pd.DataFrame(app_lag.week.unique()).rename(columns = {0:'week'}), app_lag[['suite_nps', 'suite_count']]], axis = 1)

#create a week block in this df to use as a primary key in our merge
week_df['week_block'] = [x for x in list(range(1, 81, 1))]

#merge the two
summs = week_df.merge(summs, on = 'week_block')

#convert week to datetime so format is kept when sending to csv
summs['week'] = pd.to_datetime(summs['week'])

#keep only the best fitting model for each week, if there is a tie between AIC and BIC, just keep the model with the least amount of lags
apps = ['excel_nps', 'outlook_nps', 'pp_nps', 'word_nps']

#create empty dict to store dfs
app_final_results = {}

#loop through apps and create final results dfs
for app in apps:
    
    app_final_results[app] = pd.DataFrame(summs[(summs['app'] == app) & (~summs['AIC'].astype(str).str.contains('-inf'))]\
             .groupby(['week', 'suite_nps', 'suite_count', 'week_block', 'coeff', 'lag', 'AIC']).BIC.min()).reset_index().sort_values(['week', 'lag'])\
                    .drop_duplicates(subset = ['week', 'AIC', 'BIC'], keep = 'first')\
                        .sort_values(['week', 'AIC']).drop_duplicates(subset = ['week'], keep = 'last').reset_index(drop = True)

#create 4 separate dfs for final results outputs, send to csvs
app_final_results['excel_nps'].to_csv('suite_lag_regressed_on_excel.csv', index = False, date_format = '%Y-%m-%d')
app_final_results['outlook_nps'].to_csv('suite_lag_regressed_on_outlook.csv', index = False, date_format = '%Y-%m-%d')
app_final_results['pp_nps'].to_csv('suite_lag_regressed_on_pp.csv', index = False, date_format = '%Y-%m-%d')
app_final_results['word_nps'].to_csv('suite_lag_regressed_on_word.csv', index = False, date_format = '%Y-%m-%d')